In [1]:
import math
import os
import random
from pathlib import Path
import numpy as np
import yaml
import glob
import cv2
import torch
from pathlib import Path
import pandas as pd
import os
import shutil


def make_json():
    coco_json_train = {}
    coco_json_train['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_train['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_train['annotations'] = []
    coco_json_train['categories'] = [{'supercategory': 'logitech', 'id': 0, 'name': 'logitech'},{'supercategory': 'apple', 'id': 1, 'name': 'apple'}]
    coco_json_train['images'] = []


    coco_json_val = {}
    coco_json_val['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_val['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_val['annotations'] = []
    coco_json_val['categories'] = [{'supercategory': 'logitech', 'id': 0, 'name': 'logitech'},{'supercategory': 'apple', 'id': 1, 'name': 'apple'}]
    coco_json_val['images'] = []
    
    return coco_json_train, coco_json_val


def make_dir(path):
    if not os.path.exists(path):
        print(path)
        make_dir(os.path.dirname(path))
        os.makedirs(path, exist_ok=True)

        
def myLoadImage(im_path,img_size, resize=True, augment=True) :
    im = cv2.imread(im_path)
    print(im.shape)
    h0, w0 = im.shape[:2]
    if resize : 
        r = img_size / max(h0, w0)
        if r != 1:  # if sizes are not equal
            interp = cv2.INTER_LINEAR if augment else cv2.INTER_AREA  # random.choice(self.rand_interp_methods)
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
    return im, (h0, w0), im.shape[:2] 

from PIL import Image, ImageDraw
import matplotlib.patches as patches
def myFig(img, bbox_list, drawB = True) :
    image = Image.fromarray(img,"RGB")
    draw = ImageDraw.Draw(image)

    if drawB :
        for bbox in bbox_list : 
            for i in range(len(bbox)) :
                xmin = bbox[i][1] 
                ymin = bbox[i][2]
                xmax = bbox[i][3]
                ymax = bbox[i][4]

                draw.rectangle((xmin, ymin, xmax, ymax), outline=(255,0,0), width = 1) # bounding box
#     display(image)
    return image
def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    # Convert nx4 boxes from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = w * (x[:, 0] - x[:, 2] / 2) + padw  # top left x
    y[:, 1] = h * (x[:, 1] - x[:, 3] / 2) + padh  # top left y
    y[:, 2] = w * (x[:, 0] + x[:, 2] / 2) + padw  # bottom right x
    y[:, 3] = h * (x[:, 1] + x[:, 3] / 2) + padh  # bottom right y
    return y

def convert_coordinates(x1, y1, x2, y2, img_width, img_height, cate):
    # 바운딩 박스의 중심 좌표와 너비, 높이 계산
    box_width = x2 - x1
    box_height = y2 - y1
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0
    
    # 이미지의 너비와 높이로 나누어서 정규화된 값 계산
    x_center /= img_width
    y_center /= img_height
    box_width /= img_width
    box_height /= img_height
    
    # 결과를 문자열로 반환
    return f"{cate} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

In [2]:
def test(img_size,img_list, save_img_path, save_label_path, save_annotations_path,coco_json,id_cnt,t):
    
    for i, imgs in enumerate(img_list) : 
        result_img = {}
        if len(imgs) <3 :
            continue
        for idx , img_path in enumerate(imgs) : 
            
            if 'logitech' in img_path : 
                category_id = 0
            else :
                category_id = 1
            img, (h0, w0), (h, w) = myLoadImage(img_path,img_size)

            color = random.randrange(0,255)
            back_ground = np.full((img_size * 2, img_size * 2, img.shape[2]), color, dtype=np.uint8) 
            mosaic_border = [img_size // 2, img_size // 2]

            yc, xc = (int(random.uniform(img_size, 2 *img_size)) for x in mosaic_border)  

            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            xmin, ymin = x1a, y1a

            if int(x2a - x1a)  < img_size : 
                print(f'size ERROR')
            if int(y2a - y1a)  < img_size : 
                print(f'size ERROR')
            back_ground[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]  # img4[ymin:ymax, xmin:xmax]

            padw = x1a - x1b
            padh = y1a - y1b

            label = np.array([[0. , 0.5 ,0.5 ,1. , 1. ]])

            if label.size:
                label[:, 1:] = xywhn2xyxy(label[:, 1:], w, h, padw, padh)  # normalized xywh to pixel xyxy format
                label[label < 0] = 0
                label[label > img_size*2] = img_size*2

            if idx == 0 : 
                pass
            elif idx == 1 :
                label[0][1] += img_size * 2
                label[0][3] += img_size * 2
            elif idx == 2 :
                label[0][2] += img_size * 2
                label[0][4] += img_size * 2
            elif idx == 3 :
                label[0][1] += img_size * 2
                label[0][2] += img_size * 2
                label[0][3] += img_size * 2
                label[0][4] += img_size * 2

            result_img[idx] = {'img' : back_ground, 'label':label, 'category_id':category_id}

        if len(result_img) ==4 : 

            res_img1 = np.concatenate((result_img[0]['img'],result_img[1]['img']),axis=1)
            res_img2 = np.concatenate((result_img[2]['img'],result_img[3]['img']),axis=1)
            result_image = np.concatenate((res_img1,res_img2),axis=0)
            label_list = [result_img[idx]['label'] for idx in result_img ]
        elif len(result_img) ==2 :

            result_image = np.concatenate((result_img[0]['img'],result_img[1]['img']),axis=1)
            label_list = [result_img[idx]['label'] for idx in result_img ]
        else :
            continue

        cv2.imwrite(f'{save_img_path}/{i+t}.jpg',result_image)
        height, width, _ = result_image.shape

        img_res = {'license': 1, 'file_name': f'{i+t}.jpg','coco_url': '','height': height,'width': width,'date_captured': '','flickr_url': '','id': i+t}

        coco_json['images'].append(img_res)

        with open(f'{save_label_path}/{i+t}.txt', "w") as f:
            for idx in result_img : 
                label = result_img[idx]['label'][0]
                category_id = result_img[idx]['category_id']

                x1, y1, x2, y2 = label[1],label[2],label[3],label[4]

                yolo_bbox = convert_coordinates(x1, y1, x2, y2, width, height, category_id)
                f.write(f"{yolo_bbox}\n")

                coco_bbox = [x1, y1, x2 - x1, y2 - y1]  # COCO bbox 형식으로 변환

                area = coco_bbox[2] * coco_bbox[3]
                res = {'segmentation':[] ,'area': area ,'iscrowd': 0,'image_id': i+t,'bbox': coco_bbox ,'category_id': category_id,'id': id_cnt}
                coco_json['annotations'].append(res)
                id_cnt += 1
                print(res)


    import json
    with open(save_annotations_path, "w") as f:
        json.dump(coco_json, f)
    return id_cnt



In [6]:
#폴더만들기


ori_img_path_2 = '/data/ij/Mosaic_data_augmentation_YOLO/logitech'
ori_img_path_1 = '/data/ij/Mosaic_data_augmentation_YOLO/apple'
        
root_path = '/data/ij/Mosaic_data_augmentation_YOLO/coco_ps'
if os.path.exists(root_path) : 
    shutilil.rmtree(root_path)
else :
    os.mkdir(root_path)

save_img_path = f'{root_path}/images/train2017'
save_img_path_val = f'{root_path}/images/val2017'

save_label_path = f'{root_path}/labels/train2017'
save_label_path_val = f'{root_path}/labels/val2017'
save_annotations_path = f'{root_path}/annotations/'
save_annotations_train = f'{root_path}/annotations/instances_train2017.json'
save_annotations_val = f'{root_path}/annotations/instances_val2017.json'

save_path_list = [save_img_path,save_img_path_val,save_label_path,save_label_path_val,save_annotations_path]

for save_path in save_path_list : 
    make_dir(save_path)


/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/images/train2017
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/images
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/images/val2017
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/labels/train2017
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/labels
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/labels/val2017
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/annotations/
/data/ij/Mosaic_data_augmentation_YOLO/coco_ps/annotations


In [7]:
#이미지 패스 불러오기
file_ex = "png"
im_files_1 = np.array([i for i in os.listdir(ori_img_path_1) if i.endswith(file_ex)], dtype = "object")
im_files_1 = list(ori_img_path_1 + "/" + im_files_1)

im_files_2 = np.array([i for i in os.listdir(ori_img_path_2) if i.endswith(file_ex)], dtype = "object")
im_files_2 = list(ori_img_path_2 + "/" + im_files_2)

max(len(im_files_2),len(im_files_1))
max_file = int(max(len(im_files_2),len(im_files_1)))
min_file = int(min(len(im_files_2),len(im_files_1)))
img_list = []
for idx in range(2, 2002,2):
    if idx < min_file : 
        a = im_files_2[idx-2:idx] + im_files_1[idx-2:idx]
        img_list.append(a)
    else :
        new_idx= random.randrange(0,min_file-2)
        a = im_files_2[new_idx-2:new_idx] + im_files_1[new_idx-2:new_idx]
        img_list.append(a)
        

In [8]:
#json 만들기
coco_json_train, coco_json_val = make_json()

img_size = 256

split = int(len(img_list) * 0.8)
train_img_list = img_list[:split]
val_img_list = img_list[split:]

#dataset 만들기

#train
id_cnt = 1
t = 0
id_cnt= test(img_size,train_img_list, save_img_path, save_label_path, save_annotations_train,coco_json_train, id_cnt,t)

#val
t = int(len(train_img_list))
id_cnt= test(img_size,val_img_list, save_img_path_val, save_label_path_val, save_annotations_val,coco_json_val, id_cnt,t)


(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [78.0, 37.0, 256.0, 256.0], 'category_id': 0, 'id': 1}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [663.0, 41.0, 256.0, 256.0], 'category_id': 0, 'id': 2}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [251.0, 653.0, 256.0, 256.0], 'category_id': 1, 'id': 3}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [600.0, 674.0, 256.0, 256.0], 'category_id': 1, 'id': 4}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [101.0, 72.0, 256.0, 256.0], 'category_id': 0, 'id': 5}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [713.0, 83.0, 256.0, 256.0], 'category_id': 0, 'id': 6}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [12.0, 635.0, 256.0, 256.0], 'cate

(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [46.0, 236.0, 256.0, 256.0], 'category_id': 0, 'id': 65}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [687.0, 217.0, 256.0, 256.0], 'category_id': 0, 'id': 66}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [157.0, 744.0, 256.0, 256.0], 'category_id': 1, 'id': 67}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [576.0, 654.0, 256.0, 256.0], 'category_id': 1, 'id': 68}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 17, 'bbox': [227.0, 17.0, 256.0, 256.0], 'category_id': 0, 'id': 69}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 17, 'bbox': [746.0, 196.0, 256.0, 256.0], 'category_id': 0, 'id': 70}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 17, 'bbox': [32.0, 637.0, 256.0, 256.0], 'category_id': 1, 'id': 71}
{

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [248.0, 198.0, 256.0, 256.0], 'category_id': 0, 'id': 129}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [584.0, 36.0, 256.0, 256.0], 'category_id': 0, 'id': 130}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [90.0, 520.0, 256.0, 256.0], 'category_id': 1, 'id': 131}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [585.0, 672.0, 256.0, 256.0], 'category_id': 1, 'id': 132}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 33, 'bbox': [139.0, 92.0, 256.0, 256.0], 'category_id': 0, 'id': 133}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 33, 'bbox': [687.0, 29.0, 256.0, 256.0], 'category_id': 0, 'id': 134}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 33, 'bbox': [153.0, 539.0, 256.0, 256.0], 'category_id': 1, 'id': 135}
{'segme

(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 51, 'bbox': [180.0, 200.0, 256.0, 256.0], 'category_id': 0, 'id': 205}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 51, 'bbox': [690.0, 120.0, 256.0, 256.0], 'category_id': 0, 'id': 206}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 51, 'bbox': [48.0, 735.0, 256.0, 256.0], 'category_id': 1, 'id': 207}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 51, 'bbox': [548.0, 547.0, 256.0, 256.0], 'category_id': 1, 'id': 208}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 52, 'bbox': [52.0, 43.0, 256.0, 256.0], 'category_id': 0, 'id': 209}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 52, 'bbox': [765.0, 93.0, 256.0, 256.0], 'category_id': 0, 'id': 210}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 52, 'bbox': [55.0, 548.0, 256.0, 256.0], 'category_id'

(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 70, 'bbox': [172.0, 69.0, 256.0, 256.0], 'category_id': 0, 'id': 281}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 70, 'bbox': [649.0, 146.0, 256.0, 256.0], 'category_id': 0, 'id': 282}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 70, 'bbox': [203.0, 629.0, 256.0, 256.0], 'category_id': 1, 'id': 283}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 70, 'bbox': [721.0, 716.0, 256.0, 256.0], 'category_id': 1, 'id': 284}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 71, 'bbox': [55.0, 33.0, 256.0, 256.0], 'category_id': 0, 'id': 285}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 71, 'bbox': [514.0, 56.0, 256.0, 256.0], 'category_id': 0, 'id': 286}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 71, 'bbox': [57.0, 548.0, 256.0, 256.0], 'category_id': 1, 'id': 2

...

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 924, 'bbox': [171.0, 148.0, 256.0, 256.0], 'category_id': 0, 'id': 3677}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 924, 'bbox': [661.0, 35.0, 256.0, 256.0], 'category_id': 0, 'id': 3678}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 924, 'bbox': [39.0, 627.0, 256.0, 256.0], 'category_id': 1, 'id': 3679}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 924, 'bbox': [617.0, 696.0, 256.0, 256.0], 'category_id': 1, 'id': 3680}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 925, 'bbox': [60.0, 206.0, 256.0, 256.0], 'category_id': 0, 'id': 3681}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 925, 'bbox': [627.0, 109.0, 256.0, 256.0], 'category_id': 0, 'id': 3682}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 925, 'bbox': [34.0, 696.0, 256.0, 256.0], 'category_id': 1, 'id':

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 938, 'bbox': [143.0, 4.0, 256.0, 256.0], 'category_id': 0, 'id': 3733}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 938, 'bbox': [683.0, 0.0, 256.0, 256.0], 'category_id': 0, 'id': 3734}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 938, 'bbox': [219.0, 574.0, 256.0, 256.0], 'category_id': 1, 'id': 3735}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 938, 'bbox': [659.0, 616.0, 256.0, 256.0], 'category_id': 1, 'id': 3736}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 939, 'bbox': [156.0, 161.0, 256.0, 256.0], 'category_id': 0, 'id': 3737}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 939, 'bbox': [631.0, 124.0, 256.0, 256.0], 'category_id': 0, 'id': 3738}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 939, 'bbox': [184.0, 697.0, 256.0, 256.0], 'category_id': 1, 'id':

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 956, 'bbox': [251.0, 152.0, 256.0, 256.0], 'category_id': 0, 'id': 3805}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 956, 'bbox': [671.0, 20.0, 256.0, 256.0], 'category_id': 0, 'id': 3806}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 956, 'bbox': [164.0, 647.0, 256.0, 256.0], 'category_id': 1, 'id': 3807}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 956, 'bbox': [759.0, 744.0, 256.0, 256.0], 'category_id': 1, 'id': 3808}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 957, 'bbox': [84.0, 248.0, 256.0, 256.0], 'category_id': 0, 'id': 3809}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 957, 'bbox': [652.0, 70.0, 256.0, 256.0], 'category_id': 0, 'id': 3810}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 957, 'bbox': [21.0, 540.0, 256.0, 256.0], 'category_id': 1, 'id':

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 970, 'bbox': [189.0, 37.0, 256.0, 256.0], 'category_id': 0, 'id': 3861}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 970, 'bbox': [747.0, 179.0, 256.0, 256.0], 'category_id': 0, 'id': 3862}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 970, 'bbox': [175.0, 521.0, 256.0, 256.0], 'category_id': 1, 'id': 3863}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 970, 'bbox': [661.0, 672.0, 256.0, 256.0], 'category_id': 1, 'id': 3864}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 971, 'bbox': [30.0, 249.0, 256.0, 256.0], 'category_id': 0, 'id': 3865}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 971, 'bbox': [553.0, 218.0, 256.0, 256.0], 'category_id': 0, 'id': 3866}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 971, 'bbox': [173.0, 579.0, 256.0, 256.0], 'category_id': 1, 'id

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 984, 'bbox': [103.0, 170.0, 256.0, 256.0], 'category_id': 0, 'id': 3917}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 984, 'bbox': [593.0, 225.0, 256.0, 256.0], 'category_id': 0, 'id': 3918}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 984, 'bbox': [179.0, 608.0, 256.0, 256.0], 'category_id': 1, 'id': 3919}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 984, 'bbox': [608.0, 761.0, 256.0, 256.0], 'category_id': 1, 'id': 3920}
(256, 256, 3)
(256, 256, 3)
(64, 64, 3)
(64, 64, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 985, 'bbox': [25.0, 61.0, 256.0, 256.0], 'category_id': 0, 'id': 3921}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 985, 'bbox': [608.0, 179.0, 256.0, 256.0], 'category_id': 0, 'id': 3922}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 985, 'bbox': [55.0, 584.0, 256.0, 256.0], 'category_id': 1, 'id'

In [ ]:
# myFig(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB), label_list)